In [1]:
!pip install jellyfish

In [10]:
from packages.pyspark.pyspark_pipeline import PySparkPipeline as psp
from packages.utils.transformations import MyTransformation as mtf
from packages.utils.spark_udfs import get_trimmer_udf, soundex_udf
from pyspark.sql.functions import col, array_contains
from pyspark.sql import SparkSession, functions as F
import pandas as pd
from packages.utils.generate_data_set import SyntheticMatcherDataset as smd


trimmer_udf = get_trimmer_udf(trim=1)

spark = SparkSession.builder.master("local[*]").appName("example").getOrCreate()

In [2]:
# Data for df1
data1 = [
    ["ID00005", "N039", "E298", "Q412", "V409", "R232"],  # TP1
    ["ID00009", "R822", "W179", "H017", "P323", "F298"],  # TP2
    ["ID00007", "R449", "X716", "M948", "G667", "S702"],  # TP3
    ["ID00004", "N002", "E396", "N843", "I458", "S719"],  # TP4
    ["ID10004", "N002", "E396", "N853", "I623", "S569"],  # FN1
    ["ID50004", "J547", "B222", "G492", "R551", "S490"],  # FP1
    ["IDTIE00", "N322", "K685", "T442", "C225", "W967"],  # FP-tie: this should be skipped
    ["ID50008", "N322", "K685", "T442", "C225", "W967"],  # FP2
    ["ID00000", "W815", "L281", "R155", "F768", "B914"],
    ["ID00001", "C172", "B326", "X400", "M508", "O776"],
    ["ID00002", "V683", "C265", "J127", "D589", "F482"],
    ["ID00003", "E851", "P721", "F745", "D863", "K229"],
    ["ID00016", "T873", "D670", "U046", "Z181", "X621"],
    ["ID00017", "F327", "G856", "E567", "O929", "Q721"],
    ["ID00010", "O283", "T723", "Z034", "V319", "X338"],
]

# Data for df2
data2 = [
    ["ID00005", "R746", "E298", "Q412", "L291", "R232"],  # TP1
    ["ID00009", "R822", "W179", "H017", "P323", "F298"],  # TP2
    ["ID00007", "Z011", "X716", "M948", "W967", "S702"],  # TP3
    ["ID00004", "N002", "E396", "N843", "V935", "S719"],  # TP4
    ["ID10004", "N002", "E396", "N553", "I453", "S459"],  # FN1
    ["NEW80187", "J547", "B222", "G492", "W673", "S490"],  # FP1
    ["NEW30110", "N322", "K685", "T432", "C225", "W967"],  # FP2
    ["NEW72832", "F875", "Q768", "H822", "Z154", "X678"],
    ["NEW30110", "R560", "C434", "M687", "Q689", "Q863"],
    ["NEW81243", "R762", "N687", "A109", "K476", "R637"],
    ["NEW52689", "A089", "V733", "W158", "A640", "H331"],
    ["NEW67368", "Z079", "J617", "G878", "W111", "Q500"],
    ["NEW72348", "J547", "B222", "G492", "R551", "S490"],
    ["NEW34469", "Y990", "H898", "W673", "L967", "M829"],
    ["NEW34462", "Y990", "H898", "W673", "L967", "M829"],
]

_expected = {'gt': 5, 'tp': 4, 'fp': 2, 'fn': 1}

#udfs 
# trim_udf = get_trimmer_udf(trim=0)

# Create DataFrames
columns = [0, 1, 2, 3, 4, 5]
df1 = pd.DataFrame(data1, columns=columns)
matched_df = pd.DataFrame(data2, columns=columns)

In [6]:
# Generate synthetic data
generator = smd(
    size=5_000,
    ground_truth_ratio=0.25,
    datasets_ratio=(1, 1),
    true_positive_ratio=0.75,
    threshold=3,
)
_expected = generator.expected
df1, matched_df = generator.df1, generator.df2

df1.columns = [0,1,2,3,4,5]
matched_df.columns = [0,1,2,3,4,5]

In [15]:
transformer = mtf()
match_column = 0
cols = ['1', '2', '3', '4', '5']

df1 =  pd.read_csv("data/df1.csv", usecols=[0,1,2,3,4,5], header=None)[[0,1,2,3,4,5]].sample(frac=0.1, random_state=42)
df2 =  pd.read_csv("data/df2.csv", usecols=[0,1,2,3,4,5], header=None)[[0,1,2,3,4,5]].sample(frac=0.1, random_state=42)
_expected = {"gt": 2531, "tp":1775, "fp":92, "fn":756}

transformer.apply_soundex(df1, match_column = 0)
transformer.apply_soundex(df2, match_column = 0)

matched_df = pd.concat([df2]).drop_duplicates()

spark_df1 = spark.createDataFrame(df1)
spark_df2 = spark.createDataFrame(df2)

for col_name in spark_df1.columns:
    if str(col_name) != str(match_column):
        spark_df1 = spark_df1.withColumn(str(col_name), trimmer_udf(spark_df1[col_name]))
        spark_df2 = spark_df2.withColumn(str(col_name), trimmer_udf(spark_df2[col_name]))

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [13]:
# Read CSVs with Spark
df1 = spark.read.csv("data/df1.csv", header=None, inferSchema=True).select(["_c0","_c1","_c2","_c3","_c4","_c5"])
df2 = spark.read.csv("data/df2.csv", header=None, inferSchema=True).select(["_c0","_c1","_c2","_c3","_c4","_c5"])
df3 = spark.read.csv("data/df3.csv", header=None, inferSchema=True).select(["_c0","_c1","_c2","_c3","_c4","_c5"])
df4 = spark.read.csv("data/df4.csv", header=None, inferSchema=True).select(["_c0","_c1","_c2","_c3","_c4","_c5"])
df5 = spark.read.csv("data/df5.csv", header=None, inferSchema=True).select(["_c0","_c1","_c2","_c3","_c4","_c5"])
_expected = {'gt': 25000, 'tp': 24550, 'fp': 12333, 'fn': 450}

match_column = "0"
cols = ['1', '2', '3', '4', '5']


for i in range(6):
    df1 = df1.withColumnRenamed(f"_c{i}", str(i))
    df2 = df2.withColumnRenamed(f"_c{i}", str(i))
    df3 = df3.withColumnRenamed(f"_c{i}", str(i))
    df4 = df4.withColumnRenamed(f"_c{i}", str(i))
    df5 = df5.withColumnRenamed(f"_c{i}", str(i))
    
for col_name in df1.columns:
    if col_name != match_column:
        df1 = df1.withColumn(str(col_name), soundex_udf(df1[col_name]))
        df2 = df2.withColumn(str(col_name), soundex_udf(df2[col_name]))
        df3 = df3.withColumn(str(col_name), soundex_udf(df3[col_name]))
        df4 = df4.withColumn(str(col_name), soundex_udf(df4[col_name]))
        df5 = df5.withColumn(str(col_name), soundex_udf(df5[col_name]))

        
for col_name in df1.columns:
    if col_name != match_column:
        df1 = df1.withColumn(str(col_name), trimmer_udf(df1[col_name]))
        df2 = df2.withColumn(str(col_name), trimmer_udf(df2[col_name]))
        df3 = df3.withColumn(str(col_name), trimmer_udf(df3[col_name]))
        df4 = df4.withColumn(str(col_name), trimmer_udf(df4[col_name]))
        df5 = df5.withColumn(str(col_name), trimmer_udf(df5[col_name]))        
        
    
# Combine matched_df as before
spark_df1 = df1
spark_df2 = df2.union(df3).union(df4).union(df5).dropDuplicates()

In [16]:
ground_truth_ids = spark_df1.join(spark_df2, on=["0"], how="inner").select(F.col("0").alias("df1_id")).distinct()
gt = ground_truth_ids.count()

# 1. Create entity keys for df1 and matched_df
df1_entities = spark_df1.withColumn(
    "entity_key", F.concat_ws("", *[F.col(c) for c in cols])
).select(F.col("0").alias("df1_id"), "entity_key", *cols)

matched_entities = spark_df2.withColumn(
    "entity_key", F.concat_ws("", *[F.col(c) for c in cols])
).select(F.col("0").alias("matched_id"), "entity_key", *cols)

# 2. Cross join and count matches
joined = df1_entities.alias("a").crossJoin(matched_entities.alias("b"))
match_exprs = [(F.col(f"a.{col}") == F.col(f"b.{col}")).cast("int") for col in cols]
joined = joined.withColumn("match_count", sum(match_exprs))

# 3. Filter pairs with at least 3 matching columns
filtered = joined.filter(F.col("match_count") >= 3)

# 4. For each df1_id, find the max match_count
max_match = filtered.groupBy("a.df1_id").agg(F.max("match_count").alias("max_match_count")).withColumnRenamed("df1_id", "max_df1_id")

# 5. Join back to get all (df1_id, matched_id) pairs with the max match_count
best_matches = filtered.join(
    max_match,
    (filtered["a.df1_id"] == max_match["max_df1_id"]) & (filtered["match_count"] == max_match["max_match_count"])
).select(
    F.col("a.df1_id").alias("df1_id"),
    F.col("b.matched_id").alias("matched_id"),
    F.col("match_count")
)

# 6. Group by matched_id (bucket) to collect all df1_ids assigned to it
buckets = best_matches.groupBy("matched_id").agg(
    F.collect_list("df1_id").alias("assigned_df1_ids")
)

# True Positives
tp_arr = buckets \
    .filter(array_contains(col("assigned_df1_ids"), col("matched_id"))) \
    .join(ground_truth_ids, buckets.matched_id == ground_truth_ids.df1_id, how="inner")

tp = tp_arr.count()

fp = buckets.filter(~array_contains(col("assigned_df1_ids"), col("matched_id"))).count()
fn = gt - tp

precision = tp / (tp + fp) if tp + fp > 0 else 0
recall = tp / (tp + fn) if tp + fn > 0 else 0

print(_expected)
print(f"Ground Truth (gt): {gt}")
print(f"True Positives (tp): {tp}")
print(f"False Positives (fp): {fp}")
print(f"False Negative (fn): {fn}")
print(f"precision: {precision}")
print(f"Recall: {recall}")

{'gt': 25000, 'tp': 24550, 'fp': 12333, 'fn': 450}
Ground Truth (gt): 25000
True Positives (tp): 24921
False Positives (fp): 39836
False Negative (fn): 79
precision: 0.38483870469601744
Recall: 0.99684


In [17]:
spark_df1.show()

+--------+---+---+---+---+---+
|       0|  1|  2|  3|  4|  5|
+--------+---+---+---+---+---+
|AA223369|B65|I21|E10|423|B64|
| AA41454|S53|W45|K53|236|B64|
|AA144378|C16|B63|H36|415|G65|
| AA25364|K65|E62|W32|355|E45|
|  AA8583|C63|L60|R00|236|B64|
| AA24201|J52|L54|W12|163|G65|
| AA57788|B63|D54|W45|335|F31|
| AA45000|T65|J50|L00|252|B64|
|AA205928|P20|J25|C41|216|H61|
|AA193095|P34|Y20|P52|514|M15|
|AA157869|B65|G53|F32|161|M24|
|AA182660|K53|H40|R12|316|B64|
|AA186589|R12|J16|C43|355|B64|
|AA138813|H15|A42|F00|155|G65|
|AA145313|S00|A35|L00|116|E45|
| AA72910|W25|T53|A45|526|W45|
|AA100470|W32|L60|E41|362|G12|
|AA105323|C45|B65|P40|233|S52|
|AA208694|F52|W45|J52|215|B64|
|AA215809|G63|D15|L00|614|S52|
+--------+---+---+---+---+---+
only showing top 20 rows



In [ ]:
{'gt': 2531, 'tp': 1775, 'fp': 92, 'fn': 756}
Ground Truth (gt): 25000
True Positives (tp): 24550
False Positives (fp): 12333
False Negative (fn): 450
precision: 0.6656183065368869
Recall: 0.982